In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
START_BALANCE = 10000000
WAC = 0.04
SERVICING = 0.005
NET_APR = 0.035
AMORTIZED_MONTHS = 360
PSA = 1
CDR = 0
LOSS_SEVERITY = 0
BALLOON_OR_FINAL_MONTH = 0
IO_NUM_MONTHS = 0

In [4]:
payment = -np.pmt(WAC/12, AMORTIZED_MONTHS, START_BALANCE, 0, when = 'end')
print(payment)

47741.52954654538


In [5]:
pool = pd.DataFrame(columns=['PAYMENT_IDX',
                             'START_BALANCE', 
                             'PAYMENT', 
                             'INTEREST', 
                             'PRINCIPAL', 
                             'SERVICING', 
                             'PREPAYMENT_PSA', 
                             'DEFAULT', 
                             'LOSS_SEVERITY', 
                             'ENDING_BALANCE'])

In [6]:
idx = pd.Series(np.linspace(1, AMORTIZED_MONTHS, num=AMORTIZED_MONTHS, retstep=True)[0])

In [7]:
pool['PAYMENT_IDX'] = idx

In [8]:
pool.loc[0,'START_BALANCE'] = START_BALANCE
pool.loc[0,'PAYMENT'] = payment
pool.loc[0,'INTEREST'] = START_BALANCE * NET_APR/12
pool.loc[0,'SERVICING'] = START_BALANCE * SERVICING/12
pool.loc[0,'PRINCIPAL'] = pool.loc[0,'PAYMENT'] - pool.loc[0,'INTEREST'] - pool.loc[0,'SERVICING']
pool.loc[0,'PREPAYMENT_PSA'] = START_BALANCE* ((1.0-min(100,PSA*min(0.002*(pool.loc[0,'PAYMENT_IDX']+360.0-AMORTIZED_MONTHS),0.06))) ** (1./12.)-1.) * -1
pool.loc[0,'DEFAULT'] = 0 if pool.loc[0,'PAYMENT'] == 0 else (1.-(1.-CDR/100) ** (1. / 12.))*(pool.loc[0,'START_BALANCE']-pool.loc[0,'PRINCIPAL']-pool.loc[0,'PREPAYMENT_PSA'])
pool.loc[0,'TOTAL_PRINCIPAL'] = pool.loc[0,'PRINCIPAL'] + pool.loc[0,'PREPAYMENT_PSA'] + pool.loc[0,'DEFAULT'] 
pool.loc[0,'ENDING_BALANCE'] = pool.loc[0,'START_BALANCE'] -  pool.loc[0,'PRINCIPAL'] -  pool.loc[0,'PREPAYMENT_PSA']


In [9]:
for i in range(1,360):
    pool.loc[i,'START_BALANCE'] = pool.loc[i-1,'ENDING_BALANCE'] 
    pool.loc[i,'INTEREST'] = pool.loc[i,'START_BALANCE'] * NET_APR/12
    pool.loc[i,'SERVICING'] = pool.loc[i,'START_BALANCE'] * SERVICING/12
    pool.loc[i,'PAYMENT'] = -np.pmt(WAC/12, AMORTIZED_MONTHS-pool.loc[i-1, 'PAYMENT_IDX'], pool.loc[i,'START_BALANCE'], 0)
    pool.loc[i,'PRINCIPAL'] = pool.loc[i,'PAYMENT'] - pool.loc[i,'INTEREST'] - pool.loc[i,'SERVICING']
    pool.loc[i,'PREPAYMENT_PSA'] =  (pool.loc[i,'START_BALANCE'] - pool.loc[i,'PRINCIPAL']) * ((1.0-min(100,PSA*min(0.002*(pool.loc[i,'PAYMENT_IDX']+360.0-AMORTIZED_MONTHS),0.06))) ** (1./12.)-1.) * -1
    pool.loc[i,'DEFAULT'] = 0 if pool.loc[i,'PAYMENT'] == 0 else (1.-(1.-CDR/100) ** (1. / 12.))*(pool.loc[i,'START_BALANCE']-pool.loc[i,'PRINCIPAL']-pool.loc[i,'PREPAYMENT_PSA'])
    pool.loc[i,'ENDING_BALANCE'] = pool.loc[i,'START_BALANCE'] - pool.loc[i,'PRINCIPAL'] - pool.loc[i,'PREPAYMENT_PSA'] - pool.loc[i,'DEFAULT'] 
    pool.loc[i,'TOTAL_PRINCIPAL'] =  pool.loc[i,'PRINCIPAL'] + pool.loc[i,'PREPAYMENT_PSA'] + pool.loc[i,'DEFAULT'] 


In [10]:
pool['CF'] = pool['PREPAYMENT_PSA'] + pool['INTEREST'] + pool['PRINCIPAL'] +  pool['DEFAULT']

In [11]:
pool

,PAYMENT_IDX,START_BALANCE,PAYMENT,INTEREST,PRINCIPAL,SERVICING,PREPAYMENT_PSA,DEFAULT,LOSS_SEVERITY,ENDING_BALANCE,TOTAL_PRINCIPAL,CF
0,1.00,10000000,"47,741.53","29,166.67","14,408.20","4,166.67","1,668.20",0.00,NaN,"9,983,923.61","16,076.39","45,243.06"
1,2.00,"9,983,923.61","47,733.55","29,119.78","14,453.81","4,159.97","3,329.26",0.00,NaN,"9,966,140.53","17,783.07","46,902.85"
2,3.00,"9,966,140.53","47,717.61","29,067.91","14,497.14","4,152.56","4,989.56",0.00,NaN,"9,946,653.83","19,486.70","48,554.61"
3,4.00,"9,946,653.83","47,693.69","29,011.07","14,538.18","4,144.44","6,645.81",0.00,NaN,"9,925,469.84","21,183.99","50,195.06"
4,5.00,"9,925,469.84","47,661.78","28,949.29","14,576.88","4,135.61","8,297.18",0.00,NaN,"9,902,595.79","22,874.05","51,823.34"
...,...,...,...,...,...,...,...,...,...,...,...,...
355,356.00,"40,868.07","8,255.53",119.20,"8,119.31",17.03,168.43,0.00,NaN,"32,580.34","8,287.73","8,406.93"
356,357.00,"32,580.34","8,213.07",95.03,"8,104.47",13.58,125.88,0.00,NaN,"24,349.99","8,230.35","8,325.38"
357,358.00,"24,349.99","8,170.83",71.02,"8,089.67",10.15,83.63,0.00,NaN,"16,176.69","8,173.29","8,244.31"
358,359.00,"16,176.69","8,128.81",47.18,"8,074.89",6.74,41.67,0.00,NaN,"8,060.14","8,116.56","8,163.74"


In [12]:
def attributes(df):
    cf = np.append([-START_BALANCE], np.asarray(pool['CF']))
    IRR = np.irr(cf) * 12
    AVG_LIFE_MM = ((df['PRINCIPAL'] + df['PREPAYMENT_PSA'] +  df['DEFAULT']) @ df['PAYMENT_IDX'])/START_BALANCE
    return IRR, AVG_LIFE_MM, AVG_LIFE_MM/12

In [13]:
attributes(pool)

(0.03499999999999481, 129.56167996541902, 10.796806663784919)

In [14]:
tranches = { 
            'Tranche-A': {
                            'coupon':0.0125,
                            'price':100,
                            'PercentOfPool': .4
                            },
            'Tranche-B': {
                            'coupon':0.0285,
                            'price':100,
                            'PercentOfPool': .35
                            },
            'Tranche-C': {
                            'coupon':0.035,
                            'price':100,
                            'PercentOfPool': .25
                            }
            }

In [15]:
tranches

{'Tranche-A': {'PercentOfPool': 0.4, 'coupon': 0.0125, 'price': 100},
 'Tranche-B': {'PercentOfPool': 0.35, 'coupon': 0.0285, 'price': 100},
 'Tranche-C': {'PercentOfPool': 0.25, 'coupon': 0.035, 'price': 100}}

In [16]:
d = {}
waterfallIdx = 0
residual = 0
for k,v in tranches.items():
    print(k)
    print('residual ' + str(residual))
    print('index '+str(waterfallIdx))
    df = pd.DataFrame(columns=['INTEREST', 'PRINCIPAL', 'TOTAL', 'ENDING_BALANCE'])
    df['PAYMENT_IDX'] = idx
    Balance = v['PercentOfPool'] * START_BALANCE

    df.loc[0+waterfallIdx,'INTEREST'] = Balance * v['coupon']/12
    df.loc[0+waterfallIdx,'PRINCIPAL'] = pool.loc[0+waterfallIdx,'TOTAL_PRINCIPAL'] - residual
    df.loc[0+waterfallIdx,'ENDING_BALANCE'] =  Balance - df.loc[0+waterfallIdx,'PRINCIPAL']
    df.loc[0+waterfallIdx,'TOTAL'] = df.loc[0+waterfallIdx,'PRINCIPAL'] + df.loc[0+waterfallIdx,'INTEREST']
    for i in range(waterfallIdx+1,360):
        df.loc[i,'INTEREST'] = df.loc[i-1,'ENDING_BALANCE'] * v['coupon']/12
        df.loc[i,'PRINCIPAL'] = df.loc[i-1,'ENDING_BALANCE'] if df.loc[i-1,'ENDING_BALANCE']  < pool.loc[i,'TOTAL_PRINCIPAL'] else pool.loc[i,'TOTAL_PRINCIPAL']  
        df.loc[i,'TOTAL'] = df.loc[i,'PRINCIPAL'] + df.loc[i,'INTEREST']

        end = df.loc[i-1,'ENDING_BALANCE'] - df.loc[i,'PRINCIPAL']
        if end > 0:
            df.loc[i,'ENDING_BALANCE'] = end
        else:
            df.loc[i,'ENDING_BALANCE'] = 0
            residual = df.loc[i,'PRINCIPAL']
            #print(residual)
            waterfallIdx = i
            d[k] = df
            break

Tranche-A
residual 0
index 0
Tranche-B
residual 18801.859589479172
index 84
Tranche-C
residual 5496.064065510029
index 190


In [18]:
def GetIRR(key, trancheDictionary, START_BALANCE):
    A = trancheDictionary[key]
    sb = tranches[key]['PercentOfPool'] * START_BALANCE
    total = np.asarray(A['TOTAL'])
    ret = total[~pd.isnull(total)]
    cf = np.append([-sb], ret)
    IRR = np.irr(cf) * 12
    return IRR

In [19]:
T_A_IRR = GetIRR('Tranche-A', d, START_BALANCE)
T_B_IRR = GetIRR('Tranche-B', d, START_BALANCE)
T_C_IRR = GetIRR('Tranche-C', d, START_BALANCE)

In [20]:
print('Tranche A IRR: '+str(T_A_IRR))
print('Tranche B IRR: '+str(T_B_IRR))
print('Tranche C IRR: '+str(T_C_IRR))

Tranche A IRR: 0.01249999999999929
Tranche B IRR: 0.02849999999998687
Tranche C IRR: 0.03499999999998149


# Tranch A

In [21]:
d['Tranche-A']

,INTEREST,PRINCIPAL,TOTAL,ENDING_BALANCE,PAYMENT_IDX
0,"4,166.67","16,076.39","20,243.06","3,983,923.61",1.00
1,"4,149.92","17,783.07","21,932.99","3,966,140.53",2.00
2,"4,131.40","19,486.70","23,618.10","3,946,653.83",3.00
3,"4,111.10","21,183.99","25,295.09","3,925,469.84",4.00
4,"4,089.03","22,874.05","26,963.08","3,902,595.79",5.00
...,...,...,...,...,...
355,NaN,NaN,NaN,NaN,356.00
356,NaN,NaN,NaN,NaN,357.00
357,NaN,NaN,NaN,NaN,358.00
358,NaN,NaN,NaN,NaN,359.00


# Tranch B

In [24]:
d['Tranche-B'][84:]

,INTEREST,PRINCIPAL,TOTAL,ENDING_BALANCE,PAYMENT_IDX
84,"8,312.50","25,411.03","33,723.53","3,474,588.97",85.00
85,"8,252.15","43,961.29","52,213.43","3,430,627.68",86.00
86,"8,147.74","43,711.02","51,858.76","3,386,916.67",87.00
87,"8,043.93","43,462.08","51,506.00","3,343,454.59",88.00
88,"7,940.70","43,214.46","51,155.17","3,300,240.12",89.00
...,...,...,...,...,...
355,NaN,NaN,NaN,NaN,356.00
356,NaN,NaN,NaN,NaN,357.00
357,NaN,NaN,NaN,NaN,358.00
358,NaN,NaN,NaN,NaN,359.00


# Tranch C

In [27]:
d['Tranche-C'][190:]

,INTEREST,PRINCIPAL,TOTAL,ENDING_BALANCE,PAYMENT_IDX
190,"7,291.67","18,311.85","25,603.52","2,481,688.15",191.00
191,"7,238.26","23,665.52","30,903.77","2,458,022.63",192.00
192,"7,169.23","23,523.89","30,693.12","2,434,498.74",193.00
193,"7,100.62","23,383.03","30,483.65","2,411,115.72",194.00
194,"7,032.42","23,242.92","30,275.34","2,387,872.80",195.00
...,...,...,...,...,...
355,119.20,"8,287.73","8,406.93","32,580.34",356.00
356,95.03,"8,230.35","8,325.38","24,349.99",357.00
357,71.02,"8,173.29","8,244.31","16,176.69",358.00
358,47.18,"8,116.56","8,163.74","8,060.14",359.00
